In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('./src')

import os
import cv2
import numpy as np
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import tensorflow as tf
tf.enable_eager_execution()
import matplotlib.pyplot as plt
from importlib import import_module

import configs as cfgs
from callbacks import LearningRateScheduler
from data_loader import data_loader, view_data

In [ ]:
# import experiment model
model = import_module(f'src.{cfgs.EXPERIMENT_NAME}.model').Model()

In [ ]:
# define callbacks
lr_scheduler = LearningRateScheduler()
tensorboard  = tf.keras.callbacks.TensorBoard(write_graph=False, update_freq=1)
checkpoints  = tf.keras.callbacks.ModelCheckpoint(f'results/{cfgs.EXPERIMENT_NAME}.h5', save_best_only=False)

In [ ]:
data = data_loader()

model.fit_generator(
    data,
    steps_per_epoch = cfgs.MAX_ITERATION//cfgs.MAX_EPOCH,
    epochs = cfgs.MAX_EPOCH,
    callbacks = [lr_scheduler, tensorboard, checkpoints]
)

In [ ]:
#model.save_weights('cell_detector_weights.h5')

In [ ]:
for sample in data.take(11):
    img = np.uint8(sample[0][0]*cfgs.PIXEL_STD+cfgs.PIXEL_MEAN)
    true_classes = sample[1]['classification'][0]
    true_regress = sample[1]['regression'][0]

In [ ]:
classes, regress = model.predict(sample[0])
pred_classes = np.where(classes[0]>0.3, 1, 0)
pred_regress = regress[0]

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(221)
plt.imshow(pred_classes[...,0])

plt.subplot(222)
plt.imshow(true_classes[...,0])

plt.subplot(223)
plt.imshow(pred_classes[...,1])

plt.subplot(224)
plt.imshow(true_classes[...,1])

In [ ]:
view_data(img, true_classes.numpy(), true_regress.numpy())

In [ ]:
view_data(img, pred_classes, pred_regress)

In [ ]:
from glob import glob

test_imgs = glob('/workdir/datasets/msc/UFRGS_CELL_2classes/test/imgs/*.jpg')

In [ ]:
def resize2ratio(img):
    h,w,c = img.shape
    hh = h%2**5; ww = w%2**5
    
    img = cv2.copyMakeBorder(img, 0, hh, 0, ww, cv2.BORDER_CONSTANT)
    
    return img

In [ ]:
path_img = np.random.choice(test_imgs)
img = cv2.imread(path_img)
img = resize2ratio(img)

In [ ]:
img_batch = np.float32((img-cfgs.PIXEL_MEAN)/cfgs.PIXEL_STD)[np.newaxis,...]

classes, regress = model.predict(img_batch)
pred_classes = np.where(classes[0]>0.3, 1, 0)
pred_regress = regress[0]

In [ ]:
view_data(img, pred_classes, pred_regress)

In [3]:
!python src/test_ufrgscell.py --img_dir='/workdir/datasets/msc/UFRGS_CELL_2classes/test/imgs'  \
                              --gpu=0 \
                              --image_ext='.jpg' \
                              --test_annotation_path='/workdir/datasets/msc/UFRGS_CELL_2classes/test/xml_rotdet'

Eval image exp_M3_Mauricio_mcf7_frame_17: 100%|█| 30/30 [00:09<00:00,  3.22it/s]
********************
rotation eval:
Writing normal_cell VOC resutls file
Writing mitoses VOC resutls file
Threshold:  0.5
cls : normal_cell|| Recall: 0.8004509573839336 || Precison: 0.21061999406704243|| AP: 0.6014496431194948
F1:0.6551055976661195 P:0.6972010178117048 R:0.6178128516146416
cls : mitoses|| Recall: 0.3619047584580499 || Precison: 0.1|| AP: 0.15084276473189784
F1:0.22429406623068884 P:0.22018348623853212 R:0.22857142639455785
mAP is : 0.3761462039256963


Threshold:  0.55
cls : normal_cell|| Recall: 0.7091319044992876 || Precison: 0.18659151587066153|| AP: 0.4980959888455296
F1:0.5778238355111743 P:0.5889929742388759 R:0.5670800444565051
cls : mitoses|| Recall: 0.32380952072562363 || Precison: 0.08947368421052632|| AP: 0.1381104864916151
F1:0.19354339153665293 P:0.1875 R:0.1999999980952381
mAP is : 0.31810323766857235


Threshold:  0.6000000000000001
cls : normal_cell|| Recall: 0.615558060185

| Model      | AP50 normal | AP50 mitoses | P50 normal | R50 normal | P50 mitoses | R50 mitoses |   AP50    |   AP75    |   AP50:95   |
|----------- | ----------- | ------------ | ---------- | ---------- | ----------- | ----------- |   -----   |   -----   |   -------   |
| RetinaNet  | **75.25**   |   56.47      |   81.87    | **75.87**  |   54.33     |   65.71     |   65.86   | **19.17** |   29.56     |
| R3Det      |   74.97     |   63.21      | **83.93**  |   73.61    |   66.66     |   64.76     | **69.09** |   17.91   | **30.89**   |
| DCL        |   71.81     | **63.35**    |   77.89    |   72.71    | **66.98**   | **67.62**   |   67.58   |   17.74   |   28.48     |
| CSL        |   72.42     |   53.98      |   75.37    |   73.50    |   51.93     |   63.81     |   63.20   |   18.59   |   27.00     |
| ResNet50   |   59.41     |   35.58      |   68.02    |   67.64    |   52.50     |   40.00     |   47.50   |    5.31   |   16.37     |
| Claudio    |   60.14     |   15.08      |   69.72    |   61.78    |   22.01     |   22.85     |   37.61   |    1.63   |   11.75     |